In [2]:
import pandas as pd
import numpy as np 
import pyreadstat as pt

In [73]:
def lire(fichier, annee): 
    for i in range(0, len(fichier), 1):
        df, meta = pt.read_sav(fichier)
        df['Année'] = annee
    return df

In [143]:
df1= lire("Base_SA_globale_20182019.sav", 2019)
df1
#df2= lire("cahier6_final_20172018_Prev.sav", 2018)
#df3=lire("Cahier6_def_Prev_20162017.sav", 2017)
#df4= lire("Cahier6_Prev_20152016.sav", 2016)
#df5= lire("Cahier6_Prev_20142015.sav", 2015)

,REG,PROV,COM,VIL,MEN,CENQ,NB_REPA,TYP_JR,CONS_CER1,CONS_CER2,...,S63Q3_F,S63Q3_G,S63Q3_H,S63Q3_I,S63Q3_J,S63Q3_K,strate_def,Coefftot_1819_realloue,PrimaryLast,Année
0,1.0,13.0,1.0,1.0,45.0,76.0,3.0,0.0,1.0,1.0,...,7000.0,0.0,6000.0,0.0,0.0,0.0,3.0,105.162908,1.0,2019
1,1.0,13.0,1.0,1.0,58.0,76.0,2.0,0.0,1.0,1.0,...,18000.0,9000.0,0.0,0.0,0.0,0.0,1.0,129.881678,1.0,2019
2,1.0,13.0,1.0,1.0,67.0,76.0,2.0,0.0,1.0,1.0,...,25000.0,63000.0,0.0,0.0,0.0,0.0,3.0,105.162908,1.0,2019
3,1.0,13.0,1.0,1.0,75.0,76.0,2.0,0.0,1.0,1.0,...,0.0,5000.0,0.0,0.0,0.0,0.0,3.0,105.162908,1.0,2019
4,1.0,13.0,1.0,1.0,84.0,76.0,2.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,129.881678,1.0,2019
5,1.0,13.0,1.0,1.0,91.0,3933.0,2.0,0.0,1.0,1.0,...,16000.0,30000.0,0.0,0.0,50000.0,0.0,1.0,129.881678,1.0,2019
6,1.0,13.0,1.0,8.0,2.0,74.0,3.0,0.0,1.0,1.0,...,3000.0,30000.0,0.0,0.0,0.0,0.0,3.0,121.399487,1.0,2019
7,1.0,13.0,1.0,8.0,5.0,74.0,3.0,0.0,1.0,1.0,...,10000.0,25000.0,0.0,0.0,0.0,0.0,3.0,121.399487,1.0,2019
8,1.0,13.0,1.0,8.0,131.0,74.0,3.0,0.0,1.0,1.0,...,0.0,5000.0,0.0,0.0,0.0,0.0,1.0,150.196606,1.0,2019
9,1.0,13.0,1.0,8.0,145.0,74.0,3.0,0.0,1.0,1.0,...,15000.0,0.0,0.0,0.0,0.0,0.0,3.0,121.399487,1.0,2019


In [93]:
def concat(fich1, fich2):
        donnee= fich1.merge(fich2, how="outer")
        return donnee

In [124]:
don1= concat(df1,df2)
don2= concat(don1, df3)
don3= concat(don2, df4)
donnee= concat(don3, df5)
donnee

,REG,PROV,COM,VIL,MEN,CENQ,NB_REPA,TYP_JR,CONS_CER1,CONS_CER2,...,Type_men,Classe_SDA,S62Q2,S62Q3_1,S62Q3_2,S62Q3_3,S62Q2_c,S62Q4_3_cor1,S62Q2_1,PrimaryLast3
0,1.0,13.0,1.0,1.0,45.0,76.0,3.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,13.0,1.0,1.0,58.0,76.0,2.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,13.0,1.0,1.0,67.0,76.0,2.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,13.0,1.0,1.0,75.0,76.0,2.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,13.0,1.0,1.0,84.0,76.0,2.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1.0,13.0,1.0,1.0,91.0,3933.0,2.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1.0,13.0,1.0,8.0,2.0,74.0,3.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1.0,13.0,1.0,8.0,5.0,74.0,3.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1.0,13.0,1.0,8.0,131.0,74.0,3.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1.0,13.0,1.0,8.0,145.0,74.0,3.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [128]:
#donnee.to_csv('Donnees.csv')

In [132]:
data = pd.read_csv('Donnees.csv')
data


,Unnamed: 0,REG,PROV,COM,VIL,MEN,CENQ,NB_REPA,TYP_JR,CONS_CER1,...,Type_men,Classe_SDA,S62Q2,S62Q3_1,S62Q3_2,S62Q3_3,S62Q2_c,S62Q4_3_cor1,S62Q2_1,PrimaryLast3
0,0,1.0,13.0,1.0,1.0,45.0,76.0,3.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1.0,13.0,1.0,1.0,58.0,76.0,2.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1.0,13.0,1.0,1.0,67.0,76.0,2.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1.0,13.0,1.0,1.0,75.0,76.0,2.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1.0,13.0,1.0,1.0,84.0,76.0,2.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,1.0,13.0,1.0,1.0,91.0,3933.0,2.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,1.0,13.0,1.0,8.0,2.0,74.0,3.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,1.0,13.0,1.0,8.0,5.0,74.0,3.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,1.0,13.0,1.0,8.0,131.0,74.0,3.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,1.0,13.0,1.0,8.0,145.0,74.0,3.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [151]:
col = ["S62Q2","S62Q3_1","S62Q3_2","S62Q3_3","S62Q2_c","S62Q4_3_cor1","S62Q2_1", "Année"]


coll= ["S63Q3_F","S63Q3_G","S63Q3_H","S63Q3_I","S63Q3_J","S63Q3_K", "S63Q1_F"]+col

data[coll]

,S63Q3_F,S63Q3_G,S63Q3_H,S63Q3_I,S63Q3_J,S63Q3_K,S63Q1_F,S62Q2,S62Q3_1,S62Q3_2,S62Q3_3,S62Q2_c,S62Q4_3_cor1,S62Q2_1,Année
0,7000.0,0.0,6000.0,0.0,0.0,0.0,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019
1,18000.0,9000.0,0.0,0.0,0.0,0.0,500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019
2,25000.0,63000.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019
3,0.0,5000.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019
5,16000.0,30000.0,0.0,0.0,50000.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019
6,3000.0,30000.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019
7,10000.0,25000.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019
8,0.0,5000.0,0.0,0.0,0.0,0.0,1500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019
9,15000.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019


In [139]:
data = data.rename(columns={data.columns[0]: 'index'})

In [144]:
#data.set_index(data.columns[0])

In [141]:
#data.Année

In [ ]:
#df1.corr()

In [ ]:
#df1.isna().sum()